<a href="https://colab.research.google.com/github/VMutafchiev/LLMs/blob/main/Hermes_4_IndexingWeb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -Uqqq pip --progress-bar off
!pip install -qqq langchain==0.0.228 --progress-bar off
!pip install -qqq chromadb==0.3.26 --progress-bar off
!pip install -qqq sentence-transformers==2.2.2 --progress-bar off
!pip install -qqq auto-gptq==0.2.2 --progress-bar off
!pip install -qqq einops==0.6.1 --progress-bar off
!pip install -qqq unstructured==0.8.0 --progress-bar off
!pip install -qqq transformers==4.30.2 --progress-bar off
!pip install -qqq torch==2.0.1 --progress-bar off

!pip install google-cloud-aiplatform==1.25.0
!pip install langchain==0.0.187
!pip install xmltodict==0.13.0
!pip install unstructured==0.7.0 # used by langchain
!pip install pdf2image==1.16.3 #used by langchain
!pip install requests==2.31.0
!pip install beautifulsoup4==4.12.2



  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.27.1, but you have requests 2.31.0 which is incompatible.
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 31.0 MB/s eta 0:00:00
  Obtaining dependency information for google-cloud-resource-manager<3.0.0dev,>=1.3.3 from https://files.pythonhosted.org/packages/cb/88/4fc0d6be354fbf74e8192937ef58661fce38bbf188450f952da97f3ebb47/google_cloud_resource_manager-1.10.2-py2.py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

  Obtaining dependency information for langchain==0.0.187 from https://files.pythonhosted.org/packages/53/3e/ab13413ffecea43820a573b59f313bc55ef3f80c1d98c72f92de36d881f3/langchain-0.0.187-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 960.7/960.7 kB 12.0 MB/s eta 0:00:00
  Attempting uninstall: langchain
    Found existing installation: langchain 0.0.228
    Uninstalling langchain-0.0.228:
      Successfully uninstalled langchain-0.0.228
  Obtaining dependency information for unstructured==0.7.0 from https://files.pythonhosted.org/packages/ae/f7/da376de8562a7995d3a1a4127199e03295b7decc5605b364574fa1ede5ed/unstructured-0.7.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 4.3 MB/s eta 0:00:00
  Attempting uninstall: unstructured
    Found existing installation: unstructured 0.8.0
    Uninstalling unstructured-0.8.0:
      Successfully uninstalled unstructured-0.8.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.0/143.0 kB 2.7 MB/

In [2]:
from pathlib import Path

import torch
from auto_gptq import AutoGPTQForCausalLM
from langchain.chains import ConversationalRetrievalChain
from langchain.chains.question_answering import load_qa_chain
from langchain.document_loaders import DirectoryLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import HuggingFacePipeline
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from transformers import AutoTokenizer, GenerationConfig, TextStreamer, pipeline

from transformers import AutoTokenizer, pipeline, logging
from auto_gptq import AutoGPTQForCausalLM, BaseQuantizeConfig
import argparse

from sentence_transformers import SentenceTransformer

import warnings

In [3]:
import requests
from bs4 import BeautifulSoup

def parse_sitemap(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "xml")
    urls = [element.text for element in soup.find_all("loc")]
    return urls
sitemap='https://cloud.google.com/vertex-ai/sitemap.xml'
sites = parse_sitemap(sitemap)

In [4]:
sites_filtered = [url for url in sites if '/reference/' not in url and '?hl' not in url]
sites_filtered

['https://cloud.google.com/vertex-ai/docs/general/iam-permissions',
 'https://cloud.google.com/vertex-ai/sla',
 'https://cloud.google.com/vertex-ai/docs/samples/aiplatform-sdk-embedding',
 'https://cloud.google.com/vertex-ai/docs/samples/aiplatform-get-model-evaluation-tabular-regression-sample',
 'https://cloud.google.com/vertex-ai/docs/tutorials/tabular-bq-prediction',
 'https://cloud.google.com/vertex-ai/docs/experiments/tensorboard-profiler',
 'https://cloud.google.com/vertex-ai/docs/tutorials/image-recognition-automl/training',
 'https://cloud.google.com/vertex-ai/docs/start/automl-users',
 'https://cloud.google.com/vertex-ai/docs/training/neural-architecture-search/nas-tutorials',
 'https://cloud.google.com/vertex-ai/docs/tutorials/image-recognition-custom',
 'https://cloud.google.com/vertex-ai/docs/text-data/sentiment-analysis/interpret-results',
 'https://cloud.google.com/vertex-ai/docs/core-release-notes',
 'https://cloud.google.com/vertex-ai/docs/generative-ai/code/test-code-

In [5]:
len(sites_filtered)

643

In [6]:
from langchain.document_loaders import UnstructuredURLLoader
loader = UnstructuredURLLoader(urls=sites_filtered)
documents = loader.load()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [7]:
documents[42]

Document(page_content="Home\n\nDocs\n\nVertex AI\n\nDocumentation\n\nGuides\n\nGet explanations\n\nStay organized with collections\n\nSave and categorize content based on your preferences.\n\nThis guide describes how to get explanations from a Model resource on\nVertex AI. You can get explanations in two ways:\n\nOnline explanations: Synchronous requests to the Vertex AI API, similar to\nonline predictions that return\npredictions with feature attributions.\n\nBatch explanations: Asynchronous requests to the Vertex AI API that return\npredictions with feature attributions. Batch explanations are an optional part\nof batch prediction requests.\n\nBefore you begin\n\nBefore getting explanations, you must do the following:\n\nThis step differs depending on what type of machine learning model you use:\n\n\nIf you want to get explanations from a custom-trained model,  then\nfollow either Configuring example-based explanations or\nConfiguring feature-based explanations\nto create a Model tha

In [8]:
len(documents)

643

In [9]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 1000,
    chunk_overlap  = 100)

document_chunks = text_splitter.split_documents(documents)

print(f"Number documents {len(documents)}")
print(f"Number chunks {len(document_chunks)}")

document_chunks=[f"Context: {chunk.page_content} Source: {chunk.metadata['source']}" for chunk in document_chunks]

Number documents 643
Number chunks 6269


In [10]:
!rm -rf ./documents
!mkdir ./documents

In [11]:
DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"
DEVICE

'cuda:0'

In [12]:
model_name_or_path = "TheBloke/Nous-Hermes-13B-GPTQ"
model_basename = "nous-hermes-13b-GPTQ-4bit-128g.no-act.order"

use_triton = False

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

model = AutoGPTQForCausalLM.from_quantized(model_name_or_path,
        model_basename=model_basename,
        use_safetensors=True,
        trust_remote_code=True,
        device=DEVICE,
        use_triton=use_triton,
        quantize_config=None)

In [13]:
generation_config = GenerationConfig.from_pretrained(model_name_or_path)
generation_config

GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "pad_token_id": 0,
  "transformers_version": "4.30.2"
}

In [14]:
question = (
    "Which is better Java or Python?"
)
prompt = f"""
### Instructions: {question}
### Response:
""".strip()

In [15]:
%%time
input_ids = tokenizer(prompt, return_tensors='pt').input_ids.to(DEVICE)
with torch.inference_mode():
  output = model.generate(
      inputs=input_ids,
      temperature=0.5,
      max_new_tokens=512)

CPU times: user 10.8 s, sys: 609 ms, total: 11.4 s
Wall time: 15 s


In [16]:
streamer = TextStreamer(
    tokenizer,
    skip_prompt=True,
    skip_special_tokens=True,
    use_multiprocessing=False
)

In [17]:
# Inference can also be done using transformers' pipeline
# Prevent printing spurious transformers error when using pipeline with AutoGPTQ
logging.set_verbosity(logging.CRITICAL)

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=2048,
    temperature=0,
    top_p=0.9,
    repetition_penalty=1.2,
    generation_config=generation_config,
    streamer=streamer,
    batch_size=1,
)

In [18]:
llm = HuggingFacePipeline(pipeline=pipe)
responce = llm(prompt)

Both Java and Python are popular programming languages with their own strengths and weaknesses. It ultimately depends on the specific use case, personal preference, and expertise of the programmer.


In [19]:
model = SentenceTransformer("embaas/sentence-transformers-multilingual-e5-base")

In [31]:
embeddings = HuggingFaceEmbeddings(
    model_name='embaas/sentence-transformers-multilingual-e5-base',
    model_kwargs={"device":DEVICE}
)

In [26]:
questions_dir = Path("documents")

In [27]:
DEFAULT_TEMPLATE = """
### Instructions: You are thinkorswim chat agent with a name: Lucy an A.I TLC chat agent, that is talking to a customer.
Use only the chat history, and the following information{context}.
to answer in a helpful and polite manner to the question. If you don't know the answer - make an excuse and say that you don't know.
Keep your replies short, compassionate and informative.
{chat_history}
### Input: {question}
### Response:
""".strip()

class Chatbot:
    def __init__(
        self,
        text_pipeline: HuggingFacePipeline,
        embeddings: HuggingFaceEmbeddings,
        documents_dir: Path,
        prompt_template: str = DEFAULT_TEMPLATE,
        verbose: bool = False,
    ):
        prompt = PromptTemplate(
            input_variables=["context", "question", "chat_history"],
            template = prompt_template,
        )
        self.chain = self._create_chain(text_pipeline, prompt, verbose)
        self.db = self._embed_data(documents_dir, embeddings)

    def _create_chain(
        self,
        text_pipeline: HuggingFacePipeline,
        prompt: PromptTemplate,
        verbose: bool = False,
    ):
        memory = ConversationBufferMemory(
            memory_key = "chat_history",
            human_prefix = "### Input",
            ai_prefix = "### Response",
            input_key = "question",
            output_key = "output_text",
            return_messages=False,
        )
        return load_qa_chain(
            text_pipeline,
            chain_type="stuff",
            prompt=prompt,
            memory=memory,
            verbose=verbose,
        )

    def _embed_data(
        self, documents_dir: Path, embeddings: HuggingFaceEmbeddings
      ) -> Chroma:
          loader = DirectoryLoader(documents_dir, glob="**/*txt")
          documents = loader.load()
          text_splitter = CharacterTextSplitter(chunk_size=512, chunk_overlap=0)
          texts = text_splitter.split_documents(documents)
          return Chroma.from_documents(texts, embeddings)

    def __call__(self, user_input: str) -> str:
        docs = self.db.similarity_search(user_input)
        return self.chain.run({"input_documents": docs, "question": user_input})

In [37]:
import time
import pandas as pd
import json
import os

def handle_quota_errors(func, *args, retry_delay=5,backoff_factor=2, **kwargs):
    retries = 0

    try:
      return func(*args, **kwargs)
    except Exception as e:
      print(f"error: {e}")
      retries += 1
      wait = retry_delay * (backoff_factor ** retries)
      time.sleep(wait)
      print("wait for {wait} seconds")


embeddings = HuggingFaceEmbeddings(
    model_name='embaas/sentence-transformers-multilingual-e5-base',
    model_kwargs={"device":DEVICE}
)

df = pd.DataFrame(document_chunks, columns =['text'])

index_embeddings = []

for index, doc in df.iterrows():
  print(f"Get embedding and write document for document {index} of {len(df)-1}")
  embedding = handle_quota_errors(embeddings.embed_query, doc['text'])

  if embedding is not None:

    doc_id=f"{index}.txt"
    embedding_dict = {
              "id": doc_id,
              "embedding": [str(value) for value in embedding],
    }
    index_embeddings.append(json.dumps(embedding_dict) + "\n")

    doc_id = f"{index}.txt"
file_path = os.path.join("documents", doc_id)

# Open the file in UTF-8 mode and use 'newline' parameter to handle newline characters
with open(file_path, "w", encoding="utf-8", newline="\n") as document:
    document.write(doc['text'])


with open("embeddings.json", "w") as f:
    f.writelines(index_embeddings)

Streaming output truncated to the last 5000 lines.
Get embedding and write document for document 1269 of 6268
Get embedding and write document for document 1270 of 6268
Get embedding and write document for document 1271 of 6268
Get embedding and write document for document 1272 of 6268
Get embedding and write document for document 1273 of 6268
Get embedding and write document for document 1274 of 6268
Get embedding and write document for document 1275 of 6268
Get embedding and write document for document 1276 of 6268
Get embedding and write document for document 1277 of 6268
Get embedding and write document for document 1278 of 6268
Get embedding and write document for document 1279 of 6268
Get embedding and write document for document 1280 of 6268
Get embedding and write document for document 1281 of 6268
Get embedding and write document for document 1282 of 6268
Get embedding and write document for document 1283 of 6268
Get embedding and write document for document 1284 of 6268
Get e

In [38]:
chatbot = Chatbot(llm, embeddings, "./documents/")

In [40]:
warnings.filterwarnings("ignore", category=UserWarning)

while True:
    user_input = input("Question: ")
    if user_input.lower() in ["bye", "goodbye", "see ya", "thank you bye"]:
        break
    answer = chatbot(user_input)
    print()


Question: bye
